In [2]:
!python --version

Python 3.12.7


In [1]:
# Load model directly
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("SSEF-HG-AC/distilbert-uncased-finetuned-cyberbullying")
model = AutoModelForSequenceClassification.from_pretrained("SSEF-HG-AC/distilbert-uncased-finetuned-cyberbullying")

In [3]:
import pandas as pd
import numpy as np

In [5]:
br_data = pd.read_csv("E:/Docs/UGA/Dr. Itai Himelboim/Brandwatch/Complete Data/brandwatch_data.csv",low_memory=False)

In [7]:
br_data = br_data[['Title', 'Sentiment', 'Emotion', 'Region', 'Gender', 'Impressions']]
br_data.head()

,Title,Sentiment,Emotion,Region,Gender,Impressions
0,@realest_lauren @KellyScaletta Close our borde...,negative,Anger,Florida,male,358
1,@wow36932525 Didn't Michael Herrera talk about...,negative,NaN,Oregon,unknown,280
2,@KatiePhang FOR ALL OF YOU ILLITERATE DUMMIES ...,negative,Joy,Texas,unknown,5
3,@vickiringer The border patrol couldn’t legall...,negative,Sadness,California,male,75
4,@tuszyb Another dumbed down individual. THIS H...,negative,NaN,Texas,unknown,5


In [23]:
from tqdm import tqdm
texts = br_data['Title'].tolist()
model.eval()
# Tokenize the text
# inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Function to process a batch of texts
def classify_batch(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=1)
    return predictions.tolist()

# Process data in batches
batch_size = 32  # Adjust based on your system's memory
predictions = []
for i in tqdm(range(0, len(br_data), batch_size)):
    batch_texts = br_data['Title'][i:i+batch_size].tolist()
    batch_predictions = classify_batch(batch_texts)
    predictions.extend(batch_predictions)
# Run the model on the tokenized input
# with torch.no_grad():
#     outputs = model(**inputs)
#     predictions = torch.argmax(outputs.logits, dim=1)

# Add predictions to the DataFrame
br_data['Cyberbullying Prediction'] = ["Cyberbullying" if pred == 1 else "Non-Cyberbullying" for pred in predictions.tolist()]

# Display the results for the first 10 rows
br_data.head()

100%|████████████████████████████████████████████████████████████████████████████| 2333/2333 [2:57:24<00:00,  4.56s/it]


AttributeError: 'list' object has no attribute 'tolist'

In [24]:
br_data['Cyberbullying Prediction'] = ["Cyberbullying" if pred == 1 else "Non-Cyberbullying" for pred in predictions]
br_data.head()

,Title,Sentiment,Emotion,Region,Gender,Impressions,Cyberbullying Prediction
0,@realest_lauren @KellyScaletta Close our borde...,negative,Anger,Florida,male,358,Cyberbullying
1,@wow36932525 Didn't Michael Herrera talk about...,negative,NaN,Oregon,unknown,280,Cyberbullying
2,@KatiePhang FOR ALL OF YOU ILLITERATE DUMMIES ...,negative,Joy,Texas,unknown,5,Cyberbullying
3,@vickiringer The border patrol couldn’t legall...,negative,Sadness,California,male,75,Cyberbullying
4,@tuszyb Another dumbed down individual. THIS H...,negative,NaN,Texas,unknown,5,Non-Cyberbullying


In [26]:
br_data.to_csv("distilbert_brandwatch.csv", index=False)